# Initial Data Exploration

In [94]:
import pandas as pd
import bibleAnalysisFunctions as bf

### Ingest Bible
Takes in American KJV and splits the verse reference into book, chapter, verse columns

In [95]:
bible_columns = ['BookChapterVerse','VerseText']
akjv = pd.read_excel('akjv.xlsx').drop([0]).reset_index().drop(columns='index')
akjv.columns=bible_columns
bookChapterVerse = akjv.BookChapterVerse
books, chapters, verses = bf.GetVerseReferences(bookChapterVerse)
akjv['Books'] = books
akjv['Chapters'] = chapters
akjv['Verses'] = verses


### Verse Analysis Organization
Put every word in every verse into its own column.

In [96]:
maxVerseLength = bf.GetMaxVerseLength(akjv.VerseText)

AttributeError: module 'bibleAnalysisFunctions' has no attribute 'GetMaxVerseLength'

Code below has been converted to function in functions file, but that doesn't seem to be accessible currently for some reason per the error message above.

In [82]:
#First, find the max verse length. This will be the number of columns created
verseTexts = akjv['VerseText']
lenDict = {}
counter = 0
for ele in verseTexts:
    temp_list = ele.split(' ') 
    lenDict[counter] = len(temp_list)
    counter = counter + 1
max(lenDict.values())

90

Code to import other bibles; will uniformly apply transformations to these once they have been worked through on the ajkv bible above.

In [10]:
asv = pd.read_excel('asv.xlsx')
bsb = pd.read_excel('bsb.xlsx')
cpdv = pd.read_excel('cpdv.xlsx')
kjv = pd.read_excel('kjv.xlsx')
ylt = pd.read_excel('ylt.xlsx')